# Deep Q-Learning

In this notebook, you will implement a deep Q-Learning reinforcement algorithm. The implementation borrows ideas from both the original DeepMind Nature paper and the more recent asynchronous version:<br/>
[1] "Human-Level Control through Deep Reinforcement Learning" by Mnih et al. 2015<br/>
[2] "Asynchronous Methods for Deep Reinforcement Learning" by Mnih et al. 2016.<br/>

In particular:
* We use separate target and Q-functions estimators with periodic updates to the target estimator. 
* We use several concurrent "threads" rather than experience replay to generate less biased gradient updates. 
* Threads are actually synchronized so we start each one at a random number of moves.
* We use an epsilon-greedy policy that blends random moves with policy moves.
* We taper the random search parameter (epsilon) and the learning rate to zero during training.

This gives a simple and reasonably fast general-purpose RL algorithm. We use it here for the Cartpole environment from OpenAI Gym, but it can easily be adapted to others. For this notebook, you will implement 4 steps:

1. The backward step for the Q-estimator
2. The $\epsilon$-greedy policy
3. "asynchronous" initialization 
4. The Q-learning algorithm

To get started, we import some prerequisites.

In [1]:
import BIDMat.{CMat,CSMat,DMat,Dict,FMat,FND,FFilter,Filter,GFilter,GMat,GDMat,GIMat,GLMat,GSMat,GSDMat,GND,HMat,IDict,Image,IMat,LMat,Mat,SMat,SBMat,SDMat,TMat}
import BIDMat.MatFunctions._
import BIDMat.SciFunctions._
import BIDMat.Solvers._
import BIDMat.JPlotting._
import BIDMach.Learner

import BIDMach.models.{Click,FM,GLM,KMeans,KMeansw,LDA,LDAgibbs,Model,NMF,SFA,RandomForest,SVD}
import BIDMach.networks.{Net}
import BIDMach.datasources.{DataSource,MatSource,FileSource,SFileSource}
import BIDMach.datasinks.{DataSink,MatSink}
import BIDMach.mixins.{CosineSim,Perplexity,Top,L1Regularizer,L2Regularizer}
import BIDMach.updaters.{ADAGrad,Batch,BatchNorm,Grad,IncMult,IncNorm,Telescoping}
import BIDMach.causal.{IPTW}
import BIDMach.rl.ALE

Mat.checkMKL(false)
Mat.checkCUDA
Mat.setInline


1 CUDA device found, CUDA version 8.0


In [2]:
val ale = new ALE
ale.getFloat("repeat_action_probability")

0.25

The block below lists some parameters you can tune. They should be self-explanatory. They are currently set to train CartPole-V0 to a "solved" score (> 195) most of the time. 

In [3]:
val nsteps = 20000001                    // Number of steps to run (game actions per environment)
val npar = 16                            // Number of parallel environments
val target_window = 100                  // Interval to update target estimator from q-estimator
val discount_factor = 0.99f              // Reward discount factor
val printsteps = 10000                   // Number of steps between printouts
val render = false                       // Whether to render an environment while training

val epsilon_start = 0.5f                 // Parameters for epsilon-greedy policy: initial epsilon
val epsilon_end = 0.1f                  // Final epsilon
val neps = (0.9*nsteps).toInt            // Number of steps to decay epsilon

//val learning_rate = 5e-4f
val learning_rate = 3e-6f                // Initial learning rate
val lr_end = learning_rate               // Final learning rate
val nlr = neps                           // Steps to decay learning rate
val gsq_decay = 0.99f                   // Decay factor for RMSProp
val momentum_decay = 0.9f
val gclip = 1f
val rmseps = 1e-5f
val rmsevery = 5

val nhidden = 16                         // Number of hidden layers for estimators
val nhidden2 = 32
val nhidden3 = 256

val init_moves = 10000                   // Upper bound on random number of moves to take initially
val nwindow = 4                          // Sensing window = last n images in a state

4

Below are environment-specific parameters. The function "preprocess" should process an observation returned by the environment into a vector for training. For CartPole we simply append a 1 to implement bias in the first layer. 

For visual environments you would typically crop, downsample to 80x80, set color to a single bit (foreground/background), and flatten to a vector. 

*nfeats* is the dimension of the vector output by *preprocess*.

In [4]:
val game_bin="/code/ALE/roms/Pong.bin"                 // Model type and action definitions
val VALID_ACTIONS = irow(0, 3, 4)
val nactions= VALID_ACTIONS.length
val nfeats = 80*80  
val height = 80
val width = 80

def preprocess(I:FND):FMat = {
//  Preprocess Pong game frames into vectors.
//  Input:
//    - (3,160,210) uint8 frame representing Pong game screen.
//  Returns:
//    - Downsampled (DxD) matrix of 0s and 1s, "raveled" into a 1-D vector.
    var i = 0;
    val res = zeros(80*80,1)
    while (i < 80) {
        var j = 0;
        while (j < 80) {
            val x = I.data(3*(j*2 + 160 * (i*2 + 35)));
            res.data(j + 80*i) = {if (x == 144f || x == 109f) 0f else {if (x != 0f) 1f else 0f}};
            j += 1;
        }
        i += 1;
    }
    res
}

def preprocess(I:Array[Byte]):FMat = {
//  Preprocess Pong game frames into vectors.
//  Input:
//    - (3,160,210) uint8 frame representing Pong game screen.
//  Returns:
//    - Downsampled (DxD) matrix of 0s and 1s, "raveled" into a 1-D vector.
    var i = 0;
    val res = zeros(80*80,1)
    while (i < 80) {
        var j = 0;
        while (j < 80) {
            val x = I(j*2 + 160 * (i*2 + 35));
            res.data(j + 80*i) = {if (x == 34) 0f else {if (x != 0) 1f else 0f}};
            j += 1;
        }
        i += 1;
    }
    res
}

Here is the Q-estimator class. We use two instances of this class, one for the target estimator, and one for the Q-estimator. The Q function is normally represented as a scalar $Q(x,a)$ where $x$ is the state and $a$ is an action. For ease of implementation, we actually estimate a vector-valued function $Q(x,.)$ which returns the estimated reward for every action. The model here has just a single hidden layer:

<pre>
Input Layer (nfeats) => FC Layer => RELU => FC Layer => Output (naction values)
</pre>

## 1. Implement Q-estimator gradient
Your first task is to implement the
<pre>Estimator.gradient(s, a, y)</pre>
method for this class. **gradient** should compute the gradients wrt weight arrays W1 and W2 into
<pre>self.grad['W1']
self.grad['W2']</pre>
respectively. Both <code>a</code> and <code>y</code> are vectors. Be sure to update only the output layer weights corresponding to the given action vector. 

In [5]:
import scala.collection.mutable.HashMap;

class Estimator(nhidden:Int, nhidden2:Int, nhidden3:Int, nactions:Int) {

//        """ Create model matrices, and gradient and squared gradient buffers"""
    val model = new HashMap[String, FFilter]();                 // Store model params in Filters, even for FC layers
    val grad = new HashMap[String, FFilter]();
    val gradsq = new HashMap[String, FFilter]();
    model("W1") = FFilter.FFilter2Ddn(7,7,nwindow,nhidden,4,3);                         // First conv layer
    model("W1")(?) = normrnd(0,1,49*nwindow*nhidden,1) / math.sqrt(49*nwindow).toFloat  // "Xavier" initialization
    model("W2") = FFilter.FFilter2Ddn(3,3,nhidden,nhidden2,2,1);                        // Second conv layer
    model("W2")(?) = normrnd(0,1,9*nhidden*nhidden2,1) / math.sqrt(9*nhidden).toFloat
    model("W3") = FFilter.FFilter2Ddn(10,10,nhidden2,nhidden3,1,0);                     // This is actually an FC layer
    model("W3")(?) = normrnd(0,1,10*10*nhidden2*nhidden3,1) / math.sqrt(10*10*nhidden2).toFloat
    model("W4") = FFilter.FFilter2Ddn(1,1,nhidden3,nactions,1,0);                       // Second FC layer
    model("W4")(?) = normrnd(0,1,nactions*nhidden3,1) / math.sqrt(nhidden3).toFloat
    
    
    for ((k,v) <- model) {
        grad.put(k, v.copy);
        gradsq.put(k, v.copy);
        grad(k).clear
        gradsq(k).clear
    } 

    def _forward(s:FND):(FMat, FND, FND, FMat) = {
//        """ Run the model forward given a state as input.
//    returns action predictions and the hidden state"""        
        val h = model("W1") * s                  // Forward convolution, h is a tensor
        h ~ h *@ (h>=0)                          // ReLU nonlinearity
        val h2 = model("W2") * h                 // Forward convolution, h2 is a tensor
        h2 ~ h2 *@ (h2 >= 0);
        val h3 = model("W3").asMat ^* h2.asMat;  // FC matrix multiply, h3 is a matrix
        h3 ~ h3 *@ (h3 >= 0);                   
        val rew = model("W4").asMat ^* h3;       // Final FC layer, rew is a matrix
        (rew, h, h2, h3)
    }
    
    def forward(s:FMat):(FMat, FND, FND, FMat) = {
        val ss = s.asFND(height,width,nwindow,npar).transpose(2,0,1,3);  // Fix input state tensor format
        _forward(ss);
    }
    
    
    def predict(s:FMat):FMat = {
//        """ Predict the action rewards from a given input state"""
        val (rew, h, h2, h3) = forward(s)
        rew
    }
    
    var selector:IMat = irow(0->npar)*nactions
              
    def gradient(s:FMat, a:IMat, y:FMat):Float = {
//        """ Given a state s, action a and target y, compute the model gradients"""
        val ss = s.asFND(height,width,nwindow,npar).transpose(2,0,1,3);  // Fix the input state tensor format
        val (rew, h, h2, h3) = _forward(ss);                             // Run forward
        val dout = y - rew(a + selector);                                // Get the residuals for the actions a
        val arew = zeros(rew.nrows, rew.ncols);                      
        arew(a + selector) = dout                                        // The gradient for the actions a
        
        grad("W4").asMat ~ grad("W4").asMat + (h3 *^ arew);              // Backward model gradient (FC)
        val dh3 = (model("W4").asMat * arew);                            // Backward data gradient (FC)
        dh3 ~ dh3 *@ (h3 > 0)                                            // Map gradients through the RELU
        
        grad("W3").asMat ~ grad("W3").asMat + (h2.asMat *^ dh3);         // Backward model gradient (FC)
        val dh2 = (model("W3").asMat * dh3).asFND(nhidden2,10,10,npar);  // Backward data gradient (FC)
        dh2 ~ dh2 *@ (h2 > 0)        
        
//        grad("W2") ~ grad("W2") + (dh2 *^ h);
        grad("W2").convolveM(h, dh2, false);                             // Backard model gradient (conv)
        val dh = model("W2") ^* dh2                                      // Backward data gradient (conv)
        dh ~ dh *@ (h > 0)
        
//        grad("W1") ~ grad("W1") + (dh *^ s);
        grad("W1").convolveM(ss, dh, false);                             // Backward model gradient (conv)

        sqrt((dout dotr dout)/dout.length).v                             // return the RMS residual error. 
    }
    
    def msprop(learning_rate:Float, decay_rate:Float) = {                // MSprop, i.e. RMSprop without the square root.
//        """ Perform model updates from the gradients using MSprop"""
        for ((k,v) <- model) {
            val g = grad(k).data;
            val gsq = gradsq(k).data;
            val m = model(k).data;
            val len = grad(k).length;
            var i = 0;
            while (i < len) {
                val gi = math.min(gclip, math.max(-gclip, g(i)));
                gsq(i) = decay_rate * gsq(i) + (1-decay_rate) * gi * gi;
                m(i) += learning_rate * gi / (gsq(i) + rmseps);
                g(i) = 0;             
                i += 1;
            }
        }
    }
}

## 2. Implement $\epsilon$-Greedy Policy

An $\epsilon$-Greedy policy should:
* with probability $\epsilon$ take a uniformly-random action.
* otherwise choose the best action according to the estimator from the given state.

The function below should implement this policy. For each input state, it should return a (column) vector of size nactions which are the probabilities of taking each action. Thus, the probabilities of non-optimal actions should be $\epsilon/{\rm nactions}$ and the probability of the best action should be $1-\epsilon+\epsilon/{\rm nactions}$.

Since the function processes batches of states, the input <code>state</code> is a <code>nfeatures x nbatch</code> matrix, and the returned value should be a <code>nactions x nbatch</code> matrix. 

In [6]:
val aselector = irow(0->npar)*nactions

def policy(estimator:Estimator, state:FMat, epsilon:Float):FMat = {
//    """ Take an estimator and state and predict the best action.
//    For each input state, return a vector of action probabilities according to an epsilon-greedy policy"""
    val A = ones(nactions, npar) * (epsilon / nactions)
    val q_values = estimator.predict(state)
    val (_,best_action) = maxi2(q_values)
    A(best_action + aselector) = A(best_action + aselector) + (1f - epsilon)
    A
}

This routine copies the state of one estimator into another. Its used to update the target estimator from the Q-estimator.

In [7]:
def update_estimator(to_estimator:Estimator, from_estimator:Estimator, window:Int, istep:Int) = {
//    """ every <window> steps, Copy model state from from_estimator into to_estimator"""
    if (istep % window == 0) {
        for ((k,v) <- from_estimator.model) {
            to_estimator.model(k) <-- from_estimator.model(k);
        }
    }
}

## 3. Implement "Asynchronous Threads"

Don't try that in Python!! Actually all we do here is create an array of environments and advance each one a random number of steps, using random actions at each step. Later on we will make *synchronous* updates to all the environments, but the environments (and their gradient updates) should remain uncorrelated. This serves the same goal as asynchronous updates in paper [2], or experience replay in paper [1].

In [8]:
// Create estimators
val q_estimator = new Estimator(nhidden, nhidden2, nhidden3, nactions)
val target_estimator = new Estimator(nhidden, nhidden2, nhidden3, nactions)

// The epsilon and learning rate decay schedules
// val epsilons = np.linspace(epsilon_start, epsilon_end, neps)
val epsilons = epsilon_start / (1f + row(0->neps)/(neps*epsilon_end/epsilon_start))
val learning_rates = learning_rate - row(0 -> nlr) * ((lr_end - learning_rate) / nlr)

// Initialize the games
print("Initializing games...")
val envs = new Array[ALE](npar)
val state = zeros(nfeats * nwindow, npar)
var total_time=0f
var total_steps=0
var total_epochs = 0

import java.util.Random
val rn = new Random

tic
for (i <- 0 until npar) {
    envs(i) = new ALE
    envs(i).setInt("random_seed", i)
    envs(i).loadROM(game_bin)

    val nmoves = rn.nextInt(init_moves - nwindow) + nwindow
    for (j <- 0 until nmoves) {   
        val action = VALID_ACTIONS(rn.nextInt(nactions))
        val (obs, reward, done) = envs(i).step(action)
        total_steps += 1;
        if (nmoves - j <= nwindow) {
            val k = nwindow - nmoves + j;
            state((k*nfeats)->((k+1)*nfeats), i) = preprocess(obs)
        }
        if (done) {
            envs(i).reset() 
            total_epochs += 1
        }
    }
}

total_time = toc     
println("%d steps, %d epochs in %5.4f seconds at %5.4f msecs/step" format(
    total_steps, total_epochs, total_time, 1000f*total_time/total_steps))


Initializing games...61908 steps, 48 epochs in 75.4340 seconds at 1.2185 msecs/step


Select integer actions using the probability distribution in each column of <code>probs</code>

In [9]:
def random_choices(probs:FMat):IMat = {
    val result = izeros(1, probs.ncols);
    var i = 0;
    while (i < probs.ncols) {
        val r = rn.nextFloat();
        var j = 0;
        var cumprob = probs(0, i);
        while (r > cumprob && j+1 < probs.length) {
            j += 1;
            cumprob += probs(j, i);
        }
        result(i) = j;
        i += 1
    }
    result
}

## 4. Implement Deep Q-Learning
In this cell you actually implement the algorithm. We've given you comments to define all the steps. You should also add book-keeping steps to keep track of the loss, reward and number of epochs (where env.step() returns done = true). 

In [ ]:
tic
var block_loss = 0f
var block_reward = 0f
var total_reward = 0f
total_epochs = 0
var last_epochs = 0
val new_state = state.copy

val times = zeros(1,8)
val dtimes = zeros(1,7)
for (istep <- 0 until nsteps) {
//    if (render): envs[0].render()
    times(0) = toc
    val epsilon = epsilons(math.min(istep, neps-1));                                // get an epsilon for the eps-greedy policy
    val lr = learning_rates(math.min(istep, nlr-1));                                // update the decayed learning rate
    
    update_estimator(target_estimator, q_estimator, target_window, istep);          // update the target estimator if needed    
    times(1) = toc
    
    val action_probs = policy(q_estimator, state, epsilon);                         // get the next action probabilities from the policy
    times(2) = toc
                                                          
    val actions = random_choices(action_probs);                                     // Choose actions using the policy
    val (obs, rewards, dones) = ALE.stepAll2(envs, VALID_ACTIONS(actions))           // step through parallel envs
    times(3) = toc
    
    for (i <- 0 until npar) {                                                     
        val img = preprocess(obs(i));                                               // process the observation
        new_state(?,i) = state(nfeats->state.nrows,i) on img;                       // add it to buffer of last nwindow imgs        
    }    
    total_epochs += sum(dones).v.toInt
    block_reward += sum(rewards).v  
    times(4) = toc
    
    val q_values_next = target_estimator.predict(new_state);                        // predict the Q values
    times(5) = toc
    
    dones <-- (dones + (rewards != 0f) > 0f);
    val targets = rewards+discount_factor*(1f-dones) *@ maxi(q_values_next);        // compute target values   
    block_loss += q_estimator.gradient(state, actions, targets);                    // compute q-estimator gradient and return the loss
    times(6) = toc
    
    if (istep % rmsevery == 0) {
        q_estimator.msprop(lr, gsq_decay);                       // apply the gradient update
//        print("ds1=%f, ss1=%f, ds2=%f, ss2=%f\n" format(res(0,0), res(1,0), res(0,1), res(1,1)));
    }
    times(7) = toc
    
    dtimes ~ dtimes + (times(0,1->8) - times(0,0->7))
    val t = toc;
    if (istep % printsteps == 0) {
        total_reward += block_reward;
        println("step %d, time %2.1f, loss %9.8f, epochs %d, reward/epoch %6.5f, cum reward/epoch %6.5f" format(
                istep, t, block_loss/printsteps, total_epochs, block_reward/math.max(1,total_epochs-last_epochs), total_reward/math.max(1,total_epochs)))
        last_epochs = total_epochs;
        block_reward = 0f;
        block_loss = 0f;
    }
    state <-- new_state;
}
dtimes

step 0, time 0.2, loss 0.00000265, epochs 0, reward/epoch 0.00000, cum reward/epoch 0.00000
step 10000, time 414.4, loss 0.06940310, epochs 136, reward/epoch -20.68382, cum reward/epoch -20.68382
step 20000, time 832.6, loss 0.05780986, epochs 248, reward/epoch -20.22322, cum reward/epoch -20.47581
step 30000, time 1270.2, loss 0.05724451, epochs 354, reward/epoch -19.20755, cum reward/epoch -20.09604
step 40000, time 1719.7, loss 0.05864828, epochs 448, reward/epoch -19.57447, cum reward/epoch -19.98661
step 50000, time 2159.7, loss 0.05902649, epochs 539, reward/epoch -19.24176, cum reward/epoch -19.86085
step 60000, time 2610.8, loss 0.06010906, epochs 627, reward/epoch -19.11364, cum reward/epoch -19.75598
step 70000, time 3057.6, loss 0.06163091, epochs 710, reward/epoch -18.55422, cum reward/epoch -19.61549
step 80000, time 3516.2, loss 0.06001914, epochs 790, reward/epoch -18.98750, cum reward/epoch -19.55190
step 90000, time 3979.6, loss 0.06087767, epochs 872, reward/epoch -18

And animate the model's performance. 

In [ ]:
val layer="W1"
q_estimator.model(layer).dims on q_estimator.grad(layer).dims 


In [ ]:
/*state0 = state[:,0]
for i in np.arange(200):
    envs[0].render()
    preds = test_estimator.predict(state0)
    iaction = np.argmax(preds)
    obs, _, done0, _ = envs[0].step(VALID_ACTIONS[iaction])
    state0 = np.concatenate((state0[nfeats:], preprocess(obs)))
    if (done0): envs[0].reset() */
//hist(ln(abs(q_estimator.gradsq("W1")(?))),100)
//    hist(ln(abs(q_estimator.gradsq("W1")(?))),100)

So there we have it. Simple 1-step Q-Learning can solve easy problems very fast. Note that environments that produce images will be much slower to train on than environments (like CartPole) which return an observation of the state of the system. But this model can still train on those image-based games - like Atari games. It will take hours-days however. 

In [ ]:
//q_estimator.gradient(state,actions,targets)
q_estimator.model("W2").dims

In [ ]:
val f=FFilter.FFilter2Ddn(7,7,4,16,4,3);
f.dims

In [ ]:
val test_steps = 100001;
val testprintsteps = 10000;

val test_estimator = new Estimator(nfeats*nwindow, nhidden, nactions);
test_estimator.model("W1") = loadFMat("BestW1.fmat.lz4");
test_estimator.model("W2") = loadFMat("BestW2.fmat.lz4");

block_reward = 0f;
total_reward = 0f;
total_epochs = 0;
last_epochs = 0;

tic;
for (istep <- 0 until test_steps) {
    
    val action_probs = policy(test_estimator, state, 0);                    // get the next action probabilities from the policy                                                         
    val actions = random_choices(action_probs);                             // Choose actions using the policy
    val (obs, rewards, dones) = ALE.stepAll2(envs, VALID_ACTIONS(actions))  // step through parallel envs   
    for (i <- 0 until npar) {                                                     
        val img = preprocess(obs(i));                                       // process the observation
        state(?,i) = state(nfeats->state.nrows,i) on img;                   // add it to buffer of last nwindow imgs        
    }    
    total_epochs += sum(dones).v.toInt
    block_reward += sum(rewards).v  
    
    val t = toc;
    if (istep % testprintsteps == 0) {
        total_reward += block_reward;
        println("step %d, time %2.1f, epochs %d, reward/epoch %6.5f, cum reward/epoch %6.5f" format(
                istep, t, total_epochs, block_reward/math.max(1,total_epochs-last_epochs), total_reward/math.max(1,total_epochs)))
        last_epochs = total_epochs;
        block_reward = 0f;
    }
}

